In [ ]:
!gdown 1OS2Aurb2oKmooCeYwYw0cnp542t7dyHo
!gdown 1Oibb4sFzJNLt0iBWOD4LAIKaWT7cCNT1

Downloading...
From: https://drive.google.com/uc?id=1OS2Aurb2oKmooCeYwYw0cnp542t7dyHo
To: /content/test_data.csv
100% 565k/565k [00:00<00:00, 23.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Oibb4sFzJNLt0iBWOD4LAIKaWT7cCNT1
To: /content/train_data.csv
100% 2.24M/2.24M [00:00<00:00, 141MB/s]


In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
train_df = pd.read_csv("train_data.csv")
test_df = pd.read_csv("test_data.csv")

In [ ]:
train_df = train_df[train_df['confidence'] >= 0.8].reset_index(drop=True)

In [ ]:
train_df.shape

(1832, 111)

In [ ]:
test_df = test_df[test_df['confidence'] >= 0.8].reset_index(drop=True)
test_df.shape

(465, 111)

In [ ]:
X_train = train_df['clean_text'].to_numpy()
y_train = train_df['target'].to_numpy()

In [ ]:
X_test = test_df['clean_text'].to_numpy()
y_test = test_df['target'].to_numpy()

# Bag Of Words

In [ ]:
cv = CountVectorizer()
X_train_bow = cv.fit_transform(X_train).toarray()
X_test_bow = cv.transform(X_test).toarray()

In [ ]:
X_test_bow.shape

(465, 7192)

# Training-Testing Phase

In [ ]:
def training_and_prediction(training_model, X_train, X_test, y_train, y_test):
  model = training_model
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  f1score = f1_score(y_test, y_pred)
  # print("Accuracy =", accuracy)
  # print("Confusion Matrix =\n", confusion_matrix(y_test, y_pred))
  # print(classification_report(y_test, y_pred))
  return {'Accuracy' : accuracy, 'Precision': precision, 'Recall' : recall, 'F1-Score' : f1score}

In [ ]:
def get_report(y_test, y_pred):
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  f1score = f1_score(y_test, y_pred)
  return {'Accuracy' : accuracy, 'Precision': precision, 'Recall' : recall, 'F1-Score' : f1score}

## Naive Bayes

In [ ]:
training_and_prediction(MultinomialNB(alpha=0.3), X_train_bow, X_test_bow, y_train, y_test)

{'Accuracy': 0.7548387096774194,
 'Precision': 0.7333333333333333,
 'Recall': 0.825,
 'F1-Score': 0.776470588235294}

In [ ]:
training_and_prediction(GaussianNB(), X_train_bow, X_test_bow, y_train, y_test)

{'Accuracy': 0.6752688172043011,
 'Precision': 0.657243816254417,
 'Recall': 0.775,
 'F1-Score': 0.7112810707456978}

In [ ]:
training_and_prediction(BernoulliNB(alpha=0.3), X_train_bow, X_test_bow, y_train, y_test)

{'Accuracy': 0.7655913978494624,
 'Precision': 0.73992673992674,
 'Recall': 0.8416666666666667,
 'F1-Score': 0.7875243664717348}

## SVM

In [ ]:
training_and_prediction(SVC(kernel='sigmoid'), X_train_bow, X_test_bow, y_train, y_test)

{'Accuracy': 0.7505376344086021,
 'Precision': 0.748,
 'Recall': 0.7791666666666667,
 'F1-Score': 0.763265306122449}

## Random Forest

In [ ]:
training_and_prediction(RandomForestClassifier(), X_train_bow, X_test_bow, y_train, y_test)

{'Accuracy': 0.7311827956989247,
 'Precision': 0.6936026936026936,
 'Recall': 0.8583333333333333,
 'F1-Score': 0.7672253258845437}

## Logistic Regression

In [ ]:
training_and_prediction(LogisticRegression(max_iter=1000, solver='saga'), X_train_bow, X_test_bow, y_train, y_test)

{'Accuracy': 0.7677419354838709,
 'Precision': 0.764,
 'Recall': 0.7958333333333333,
 'F1-Score': 0.7795918367346939}

## Gredient Boosting

In [ ]:
training_and_prediction(XGBClassifier(), X_train_bow, X_test_bow, y_train, y_test)

{'Accuracy': 0.7569892473118279,
 'Precision': 0.7550200803212851,
 'Recall': 0.7833333333333333,
 'F1-Score': 0.7689161554192229}

## ANN

In [ ]:
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
def ann_models(num_neurons_layer1, num_neurons_layer2, dropout_rate):
    model_dense = Sequential()
    model_dense.add(Dense(num_neurons_layer1, input_dim=X_train_bow.shape[1], activation='relu'))
    model_dense.add(Dropout(dropout_rate))
    model_dense.add(Dense(num_neurons_layer2, activation='relu'))
    model_dense.add(Dense(1, activation='sigmoid'))
    model_dense.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model_dense.fit(X_train_bow, y_train, epochs=20, batch_size=32, verbose=0, callbacks=[early_stop], validation_data=(X_test_bow, y_test))
    y_pred_dense = model_dense.predict(X_test_bow)
    y_pred_dense = (y_pred_dense > 0.5).astype(int)
    return [{'num_neurons_layer1': num_neurons_layer1, 'num_neurons_layer2': num_neurons_layer2, 'dropout_rate': dropout_rate}, get_report(y_test, y_pred_dense)]

In [ ]:
for num_neurons_layer1 in [128, 64]:
  for num_neurons_layer2 in [64, 32]:
    for dropout_rate in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]:
      print(ann_models(num_neurons_layer1, num_neurons_layer2, dropout_rate))

15/15 [==============================] - 0s 5ms/step
[{'num_neurons_layer1': 128, 'num_neurons_layer2': 64, 'dropout_rate': 0.0}, {'Accuracy': 0.7612903225806451, 'Precision': 0.7698744769874477, 'Recall': 0.7666666666666667, 'F1-Score': 0.7682672233820459}]
15/15 [==============================] - 0s 5ms/step
[{'num_neurons_layer1': 128, 'num_neurons_layer2': 64, 'dropout_rate': 0.1}, {'Accuracy': 0.7634408602150538, 'Precision': 0.7480916030534351, 'Recall': 0.8166666666666667, 'F1-Score': 0.7808764940239045}]
15/15 [==============================] - 0s 4ms/step
[{'num_neurons_layer1': 128, 'num_neurons_layer2': 64, 'dropout_rate': 0.2}, {'Accuracy': 0.7634408602150538, 'Precision': 0.7801724137931034, 'Recall': 0.7541666666666667, 'F1-Score': 0.7669491525423728}]
15/15 [==============================] - 0s 4ms/step
[{'num_neurons_layer1': 128, 'num_neurons_layer2': 64, 'dropout_rate': 0.3}, {'Accuracy': 0.7483870967741936, 'Precision': 0.7142857142857143, 'Recall': 0.854166666666666